In [12]:
import requests
import h5py
import os
import cv2
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.pylab as plb
import numpy as np
import pandas as pd
from glob import glob

In [13]:
HEIGHT = 256
WIDTH = 256
CHANNELS = 3
images_path = "./images/256_ObjectCategories"
h5_file_output = "./datasets/256_images.h5"

In [28]:
def export_images():
    start = dt.datetime.now()
    dtype = h5py.special_dtype(vlen=np.dtype('uint8'))
    print("Does " + h5_file_output + "exist: " + os.path.exists(h5_file_output))

    for directory in next(os.walk(images_path))[1]:
        SOURCE_IMAGES = os.path.join(images_path, directory)
        images = glob(os.path.join(SOURCE_IMAGES, "*.jpg"))
        
        NUM_IMAGES = len(images)
        SHAPE = (HEIGHT, WIDTH, CHANNELS)
        
        if (os.path.exists(h5_file_output) == False):
            save_images_to_h5(images, directory)
        else:
            os.remove(h5_file_output)
            save_images_to_h5(images, directory)

In [26]:
def save_images_to_h5(images, directory):
    with h5py.File(h5_file_output, "w") as wf:
        with h5py.File(h5_file_output, "a") as hf:
            for i, img in enumerate(images):
                image = cv2.imread(img)
                image = cv2.resize(image, (WIDTH, HEIGHT), interpolation=cv2.INTER_CUBIC)

                X_set = hf.create_dataset(
                    name="X" + str(i) + "_" + directory,
                    data=image,
                    shape=(HEIGHT, WIDTH, CHANNELS),
                    maxshape=(HEIGHT, WIDTH, CHANNELS),
                    compression="gzip",
                    compression_opts=9
                )

                y_set = hf.create_dataset(
                    name="y" + str(i) + "_" + directory,
                    data=directory,
                    shape=(1,),
                    maxshape=(1,),
                    compression="gzip",
                    compression_opts=9
                )

In [27]:
export_images()

True
